<a href="https://colab.research.google.com/github/Panatore/RoomsClassifier/blob/main/TFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>